In [1]:
import numpy as np
import pandas as pd
import torch
import tensorly as tl
# library for TTD   (Tensorly  (search))

from tensorly.decomposition import parafac
from tensorly.decomposition import tucker
from tensorly.decomposition import tensor_train
from tensorly.decomposition import tensor_train
# different forms of tensor
from sklearn.preprocessing import MinMaxScaler
from tensorly import tt_to_tensor
from tensorly.decomposition import matrix_product_state
# tensor operations


#from math import ceil
#from google.colab import drive
#drive.mount('/content/gdrive')
import torch.nn.init as init
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader
import time
import os
import copy
import torch.nn.functional as Fun
print("PyTorch Version:", torch.__version__)
print("Torchvision Version:", torchvision.__version__)
print("GPU is available?", torch.cuda.is_available())

PyTorch Version: 1.13.1
Torchvision Version: 0.14.1
GPU is available? True


In [2]:
dtype = torch.float
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
features = list()
with open("/home/c/cl237/TenBCD/UCI HAR/data/features.txt") as f:
    features = [line.split()[1] for line in f.readlines()]
print('# of Features: {}'.format(len(features)))

# of Features: 561


## get train data

In [4]:
pd_X_train = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/X_train.txt', delim_whitespace=True, header=None)
pd_X_train.columns = features
pd_y_train = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/y_train.txt', delim_whitespace=True, names=['Activity'])

N = len(pd_X_train)

K = 6

x = pd_X_train.values
y = pd_y_train.values

X_train=torch.tensor(x,dtype = torch.float32,device=device)
y_train=torch.tensor(y,dtype = int) #dtype=torch.int,device=device)

y_train = torch.flatten(y_train)

X_train = torch.t(X_train)

y_one_hot = torch.zeros(N, K).scatter_(1, torch.reshape(y_train-1, (N, 1)), 1)
y_one_hot = torch.t(y_one_hot).to(device=device)
y_train = y_train.to(device=device)

## get test data

In [5]:
pd_X_test = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/X_test.txt', delim_whitespace=True, header=None)
pd_X_test.columns = features
pd_y_test = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/y_test.txt', delim_whitespace=True, names=['Activity'])

N_test = len(pd_X_test)

x = pd_X_test.values
y = pd_y_test.values

X_test=torch.tensor(x,dtype = torch.float32, device=device)
y_test=torch.tensor(y,dtype = int)

y_test = torch.flatten(y_test)

X_test = torch.t(X_test)

y_test_one_hot = torch.zeros(N_test, K).scatter_(1, torch.reshape(y_test-1, (N_test, 1)), 1)
y_test_one_hot = torch.t(y_test_one_hot).to(device=device)
y_test = y_test.to(device=device)

In [6]:
def updateV(U1,U2,W,b,rho,gamma): 
    _, d = W.size()
    I = torch.eye(d, device=device) #Returns a 2-D tensor with ones on the diagonal and zeros elsewhere.
    U1 = nn.ReLU()(U1)
    _, col_U2 = U2.size()
    Vstar = torch.mm(torch.inverse(rho*(torch.mm(torch.t(W),W))+gamma*I), rho*torch.mm(torch.t(W),U2-b.repeat(1,col_U2))+gamma*U1)
    # check how to get V* W*
    # torch.mm*=(input,mat2) performs a matrix multiplication of the matrices input and mat2.
    #   if input is a (n*m) tensor,mat2 is a (m*p) tensor, output will be (n*p) tensor
    return Vstar

def updateWb_org(U, V, W, b, alpha, rho): 
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse(alpha*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

def updateWb(U, V, W, b, W_tensor_rec, alpha, rho,tau): 
    W_tensor_rec = torch.as_tensor(W_tensor_rec,device=device).float()
    W_tensor2matrix = W_tensor_rec.reshape(W.shape)
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+tau*W_tensor2matrix+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse((alpha+tau)*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

# U update  E(1) in global convergent
def relu_prox(a, b, gamma, d, N):
    val = torch.empty(d,N, device=device)
    x = (a+gamma*b)/(1+gamma)
    y = torch.min(b,torch.zeros(d,N, device=device))

    val = torch.where(a+gamma*b < 0, y, torch.zeros(d,N, device=device))
    val = torch.where(((a+gamma*b >= 0) & (b >=0)) | ((a*(gamma-np.sqrt(gamma*(gamma+1))) <= gamma*b) & (b < 0)), x, val)
    val = torch.where((-a <= gamma*b) & (gamma*b <= a*(gamma-np.sqrt(gamma*(gamma+1)))), b, val)
    return val

In [9]:
########### parameter setup
#df = pd.DataFrame()
#df.to_csv('C:/Users/Mark/Desktop/rank40_100times.csv')
niter = 500
rank = 80
tau = 5
alpha = 1
gamma = 0.5
rho = 0.5

#alpha1 = alpha2 = alpha3 = alpha4 = alpha5 = alpha6 = alpha7 = alpha8 = alpha9 = alpha10 = alpha
#for rank in (35,): #(25,30,35)(100,180,220,260,300,340,380)(20,60,100,140,180,220,260,300,340,380)
#    for tau in (400,500): #(300,400,500)(10,50,100,200,300)(10,50,100,200,300)
#        for gamma in (0.5,0.8,2): #(0.5,0.8,2)(0.5,0.8)(0.5,1,1.5,2,3)
            #gamma1 = gamma2 = gamma3 = gamma4 = gamma5 = gamma
#            for rho in (0.5,0.8,2): #(0.5,0.8)(1,2)
                #rho1 = rho2 = rho3 = rho4 = rho5= rho
#                for alpha in (0.5,1,1.5,2):
#                    print ("rank=",rank, "tau=",tau, "gamma=",gamma, "rho=",rho, "alpha",alpha)
                    #print('Compression Ratio', ((1024*28*28+10*1024+(8*(rank)+32*np.square(rank))*2)/(1024*28*28+10*1024+1024*1024*2)), (8*(rank)+32*np.square(rank))*2/(1024*1024*2))
        
print ("rank=",rank, "tau=",tau, "gamma=",gamma, "rho=",rho, "alpha",alpha)

loss1 = np.empty(niter)
loss2 = np.empty(niter)
accuracy_train = np.empty(niter)
accuracy_test = np.empty(niter)
time1 = np.empty(niter)

results = torch.zeros(1, 5, niter)



for Out_iter in range(1):
    rank_initial = 400
    seed = 10 + 10*Out_iter
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.manual_seed(seed)

    d0 = 561 #561 =3*11*17
    d1 =  1024    # 2^10
    d2 =  1024  
    d3 = 1024
    d4 = 512      # 2^9
    d5 = 6 

    W1 = init.uniform_(torch.empty(d1, d0, device=device), a=-0.01, b=0.01)
    #W1 = 0.01*torch.randn(d1, d0, device=device)
    W1_torch_tensor = W1.reshape((6,22,34,2,2,2,2,2,2,2))
    W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
    factors1 = tensor_train(W1_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W1_tl_tensor_rec = tt_to_tensor(factors1)
    b1 = 0*torch.ones(d1, 1, device=device) # 0 is stable


    W2 = init.uniform_(torch.empty(d2, d1, device=device), a=-0.01, b=0.01)
    W2_torch_tensor = W2.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy())
    factors2 = tensor_train(W2_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    # we have 9 rank_initial (r1,...,r9) here plus 2 of 1s (r0 & r10).
    # factors: set of tensor cores
    # http://tensorly.org/stable/user_guide/tensor_decomposition.html
    # http://tensorly.org/stable/modules/generated/tensorly.decomposition.tensor_train.html#tensorly.decomposition.tensor_train
    W2_tl_tensor_rec = tt_to_tensor(factors2)
    # tt_to_tensor(factors):Re-assembles ‘factors’, which represent a tensor in TT format into the corresponding full tensor
    #      facros: list of 3d-arrays tt-cores           output_tensor: ndarray      
    b2 = 0*torch.ones(d2, 1, device=device)


    W3 = init.uniform_(torch.empty(d3, d2, device=device), a=-0.01, b=0.01)
    W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
    factors3 = tensor_train(W3_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W3_tl_tensor_rec = tt_to_tensor(factors3)
    b3 = 0*torch.ones(d3, 1, device=device)

    W4 = init.uniform_(torch.empty(d4, d3, device=device), a=-0.01, b=0.01)
    W4_torch_tensor = W4.reshape((4,4,4,4,4,4,4,4,4,2)) # 8 number of 4s, 2 number of 8s
    W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())
    factors4 = tensor_train(W4_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W4_tl_tensor_rec = tt_to_tensor(factors4)
    b4 = 0*torch.ones(d4, 1, device=device)


    W5 = init.uniform_(torch.empty(d5, d4, device=device), a=-0.01, b=0.01)
    b5 = 0*torch.ones(d5, 1, device=device)

    # W4 = 0.01*torch.randn(d4, d3, device=device)
    # W4_torch_tensor = W4.reshape((50,30,K))
    # W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())
    # core4, tucker_factors4 = tucker(W4_tl_tensor, rank=(40,28,K), init='svd', tol=10e-5, random_state=12345)
    # W4_tl_tensor_rec = tl.tucker_to_tensor((core4, tucker_factors4))
    # b4 = 0.1*torch.ones(d4, 1, device=device)

    U1 = torch.addmm(b1.repeat(1, N), W1, X_train)
    V1 = nn.ReLU()(U1)
    U2 = torch.addmm(b2.repeat(1, N), W2, V1)
    V2 = nn.ReLU()(U2)
    U3 = torch.addmm(b3.repeat(1, N), W3, V2)
    V3 = nn.ReLU()(U3)
    U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    V4 = nn.ReLU()(U4)
    U5 = torch.addmm(b5.repeat(1, N), W5, V4)
    V5 = U5 
    # U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    # V4 = U4



    # Iterations
    print('Train on', N, 'samples, validate on', N_test, 'samples')
    for k in range(niter):
        start = time.time()

 # update for last layer
        # update V4
        V5 = (y_one_hot + gamma*U5 + alpha*V5)/(1 + gamma + alpha)

        # update U4 
        U5 = (gamma*V5 + rho*(torch.mm(W5,V4) + b5.repeat(1,N)))/(gamma + rho)

        # update W4 and b4
        W5, b5 = updateWb_org(U5,V4,W5,b5,alpha,rho)
        # W3_torch_tensor = W3.reshape((d3,4,4,4,4,4,4))
        # W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
        # factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,1))
        # W3_tl_tensor_rec = tt_to_tensor(factors3)



  # update for 4th layer
        # update V3
        V4 = updateV(U4,U5,W5,b5,rho,gamma)

        # update U3
        U4 = relu_prox(V4,(rho*torch.addmm(b4.repeat(1,N), W4, V3) + alpha*U4)/(rho + alpha),(rho + alpha)/gamma,d4,N)

        # update W3 and b3
        W4, b4 = updateWb(U4,V3,W4,b4,W4_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W4_torch_tensor = W4.reshape((4,4,4,4,4,4,4,4,4,2))
        W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors4 = tensor_train(W4_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W4_tl_tensor_rec = tt_to_tensor(factors4)

  # update for 3nd layer
        # update V3
        V3 = updateV(U3,U4,W4,b4,rho,gamma)

        # update U3
        U3 = relu_prox(V3,(rho*torch.addmm(b3.repeat(1,N), W3, V2) + alpha*U3)/(rho + alpha),(rho + alpha)/gamma,d3,N)

        # update W3 and b3
        W3, b3 = updateWb(U3,V2,W3,b3,W3_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4))
        W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W3_tl_tensor_rec = tt_to_tensor(factors3)

 # update for 2nd layer
        # update V2
        V2 = updateV(U2,U3,W3,b3,rho,gamma)

        # update U2
        U2 = relu_prox(V2,(rho*torch.addmm(b2.repeat(1,N), W2, V1) + alpha*U2)/(rho + alpha),(rho + alpha)/gamma,d2,N)

        # update W2 and b2
        W2, b2 = updateWb(U2,V1,W2,b2,W2_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W2_torch_tensor = W2.reshape((4,4,4,4,4,4,4,4,4,4))
        W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy()) 
        factors2 = tensor_train(W2_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        W2_tl_tensor_rec = tt_to_tensor(factors2)


# update for 1st layer
        # update V1
        V1 = updateV(U1,U2,W2,b2,rho,gamma)

        # update U1
        U1 = relu_prox(V1,(rho*torch.addmm(b1.repeat(1,N), W1, X_train) + alpha*U1)/(rho + alpha),(rho + alpha)/gamma,d1,N)

        # update W1 and b1
        W1, b1 = updateWb(U1,X_train,W1,b1,W1_tl_tensor_rec, alpha,rho,tau)

        # G update
        W1_torch_tensor = W1.reshape((6,22,34,2,2,2,2,2,2,2))
        W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
        factors1 = tensor_train(W1_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        W1_tl_tensor_rec = tt_to_tensor(factors1)




 # prediction for trainning data
        a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), X_train))
        #a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
        a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
        a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_train))
        a4_train = nn.ReLU()(torch.addmm(b4.repeat(1, N), torch.as_tensor(W4_tl_tensor_rec,device=device).reshape((d4, d3)).float(), a3_train))
        #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        pred = torch.argmax(torch.addmm(b5.repeat(1, N), W5, a4_train), dim=0)
        # check argmax and addmm, dim=0

 #Prediction for test data
        a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), X_test))
        #a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
        a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
        a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_test))
        a4_test = nn.ReLU()(torch.addmm(b4.repeat(1, N_test), torch.as_tensor(W4_tl_tensor_rec,device=device).reshape((d4, d3)).float(), a3_test))
        pred_test = torch.argmax(torch.addmm(b5.repeat(1, N_test), W5, a4_test), dim=0)

        # a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), x_train))
        # a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
        # # a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_train))
        # #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        # pred = torch.argmax(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_train), dim=0)

        # a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), x_test))
        # a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
        # # a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_test))
        # pred_test = torch.argmax(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_test), dim=0)

        # a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
        # a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), W2, a1_train))

        # #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        # pred = torch.argmax(torch.addmm(b3.repeat(1, N), W3, a2_train), dim=0)

        # a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
        # a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), W2, a1_test))
        # pred_test = torch.argmax(torch.addmm(b3.repeat(1, N_test), W3, a2_test), dim=0)




   #emperical loss
        loss1[k] = gamma/2*torch.pow(torch.dist(V5,y_one_hot,2),2).cpu().numpy()
        # torch.pow:Takes the power of each element in input with exponent and returns a tensor with the result.
        # torch.disk: Returns the p-norm of (input - other)
        # Eq (5) in paper
        loss2[k] = loss1[k] + rho/2*torch.pow(torch.dist(torch.addmm(b1.repeat(1,N), W1, X_train),U1,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b2.repeat(1,N), W2, V1),U2,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b3.repeat(1,N), W3, V2),U3,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b4.repeat(1,N), W4, V3),U4,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b5.repeat(1,N), W5, V4),U5,2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V1,nn.ReLU()(U1),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V2,nn.ReLU()(U2),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V3,nn.ReLU()(U3),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V4,nn.ReLU()(U4),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V5,U5,2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W1.reshape((6,22,34,2,2,2,2,2,2,2)),torch.as_tensor(W1_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W2.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W2_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W3.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W4.reshape((4,4,4,4,4,4,4,4,4,2)),torch.as_tensor(W4_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \

        # +tau/2*torch.pow(torch.dist(W3.reshape((d3,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() 
        # +tau/2*torch.pow(torch.dist(W4.reshape((50,30,K)),torch.as_tensor(W4_tl_tensor_rec,device=device),2),2).cpu().numpy() 

        # compute training accuracy
        correct_train = pred == y_train-1
        accuracy_train[k] = np.mean(correct_train.cpu().numpy())

        # compute validation accuracy
        correct_test = pred_test == y_test-1
        accuracy_test[k] = np.mean(correct_test.cpu().numpy())

        # compute training time
        stop = time.time()
        duration = stop - start
        time1[k] = duration

        # print results
        print('Repeatition', Out_iter + 1, 'Epoch', k + 1, '/', niter, '\n', 
              '-', 'time:', time1[k], '-', 'sq_loss:', loss1[k], '-', 'tot_loss:', loss2[k], 
              '-', 'acc:', accuracy_train[k], '-', 'val_acc:', accuracy_test[k])

##############
############## compute CR
    factors1_shape=[f.shape for f in factors1]
    Sum_of_variables_factors1=sum(list(x*y*z for x,y,z in factors1_shape))
    factors2_shape=[f.shape for f in factors2]
    Sum_of_variables_factors2=sum(list(x*y*z for x,y,z in factors2_shape))
    factors3_shape=[f.shape for f in factors3]
    Sum_of_variables_factors3=sum(list(x*y*z for x,y,z in factors3_shape))
    factors4_shape=[f.shape for f in factors4]
    Sum_of_variables_factors4=sum(list(x*y*z for x,y,z in factors4_shape))

    total_variabels=Sum_of_variables_factors1+Sum_of_variables_factors2+Sum_of_variables_factors3+Sum_of_variables_factors4

    CR_1=((total_variabels)+(d4*d5))/(d0*d1+d1*d2+d2*d3+d3*d4+d4*d5)
    CR_2=(total_variabels)/(d0*d1+d1*d2+d2*d3+d3*d4)
    print("CR_1 = "+str(CR_1)+"   "+"CR_2 = " +str(CR_2))


    results[Out_iter,0,:] = torch.tensor(loss1)
    results[Out_iter,1,:] = torch.tensor(loss2)
    results[Out_iter,2,:] = torch.tensor(accuracy_train)
    results[Out_iter,3,:] = torch.tensor(accuracy_test)
    results[Out_iter,4,:] = torch.tensor(time1)
    CR=(CR_1,CR_2)

#this postion to add new row into existing table
    #df=pd.read_csv('C:/Users/Mark/Desktop/rank40_100times.csv')
    #new_row = {'rank':rank, 'CR_1':CR_1, 'CR_2':CR_2, 'tau':tau, 'gamma':gamma, 'rho':rho, 'alpha':alpha, \
    #           'loss1':loss1[niter-1], 'loss2':loss2[niter-1], 'accuracy_train':accuracy_train[niter-1], \
    #           'accuracy_test':accuracy_test[niter-1], 'time':time1[niter-1], 'seed':seed} 
    #df=df.append(new_row,ignore_index=True)
    #df.to_csv('C:/Users/Mark/Desktop/rank40_100times.csv',index=False)


filename= "UniformScaled_" + "niter_"+ str(niter) + "rank_" + str(rank) + "tau_" + str(tau) + "gamma_" + str(gamma) + \
"rho_" + str(rho) + "alpha_" + str(alpha) + ".mat"
from scipy.io import savemat
%cd /home/c/cl237/TenBCD/UCI HAR/4 hidden layers/Different Initialization/
savemat (filename, {'results': torch.Tensor.numpy(results), 'Compression Ratio':CR})
#this position to save table into matlab
#df.to_csv('C:/Users/Mark/Desktop/result_compress_123_layer_highrank.csv')
##Changing Folder
 #%cd '/content/gdrive/MyDrive/Colab/Tensor-BCD-for-DNN/Experiments/DataSaved'
 #savemat("ThreeLayer_rank160.mat", {'results': torch.Tensor.numpy(results)})    

rank= 80 tau= 5 gamma= 0.5 rho= 0.5 alpha 1
Train on 7352 samples, validate on 2947 samples
Repeatition 1 Epoch 1 / 500 
 - time: 1.2650587558746338 - sq_loss: 661.6840209960938 - tot_loss: 979.3175804925067 - acc: 0.1913764961915125 - val_acc: 0.18221920597217509
Repeatition 1 Epoch 2 / 500 
 - time: 1.2495293617248535 - sq_loss: 294.08172607421875 - tot_loss: 477.50140278600156 - acc: 0.1868879216539717 - val_acc: 0.18052256532066507
Repeatition 1 Epoch 3 / 500 
 - time: 1.2457494735717773 - sq_loss: 163.0446319580078 - tot_loss: 261.5023195575923 - acc: 0.1868879216539717 - val_acc: 0.18052256532066507
Repeatition 1 Epoch 4 / 500 
 - time: 1.2583062648773193 - sq_loss: 88.92915344238281 - tot_loss: 146.12165162945166 - acc: 0.1868879216539717 - val_acc: 0.18052256532066507
Repeatition 1 Epoch 5 / 500 
 - time: 1.2427732944488525 - sq_loss: 48.00612258911133 - tot_loss: 83.93502086866647 - acc: 0.1868879216539717 - val_acc: 0.18052256532066507
Repeatition 1 Epoch 6 / 500 
 - time: 1.

Repeatition 1 Epoch 48 / 500 
 - time: 1.229198932647705 - sq_loss: 0.0005501354462467134 - tot_loss: 0.8355491286729375 - acc: 0.8430359085963003 - val_acc: 0.8157448252460129
Repeatition 1 Epoch 49 / 500 
 - time: 1.2907612323760986 - sq_loss: 0.0005122909205965698 - tot_loss: 0.7989914854806557 - acc: 0.8495647442872688 - val_acc: 0.821174075330845
Repeatition 1 Epoch 50 / 500 
 - time: 1.2535686492919922 - sq_loss: 0.0004773948749061674 - tot_loss: 0.7580494280136918 - acc: 0.8566376496191512 - val_acc: 0.8266033254156769
Repeatition 1 Epoch 51 / 500 
 - time: 1.2322325706481934 - sq_loss: 0.0004454321460798383 - tot_loss: 0.7245077839925216 - acc: 0.8628944504896626 - val_acc: 0.835086528673227
Repeatition 1 Epoch 52 / 500 
 - time: 1.2753174304962158 - sq_loss: 0.0004158554074820131 - tot_loss: 0.6929346535907825 - acc: 0.8694232861806311 - val_acc: 0.8428910756701731
Repeatition 1 Epoch 53 / 500 
 - time: 1.2575092315673828 - sq_loss: 0.0003884805191773921 - tot_loss: 0.66449245

Repeatition 1 Epoch 94 / 500 
 - time: 1.3104820251464844 - sq_loss: 3.803599247476086e-05 - tot_loss: 0.17528392294275363 - acc: 0.9405603917301415 - val_acc: 0.9311163895486936
Repeatition 1 Epoch 95 / 500 
 - time: 1.3558707237243652 - sq_loss: 3.628957347245887e-05 - tot_loss: 0.1732549317853227 - acc: 0.9421926006528836 - val_acc: 0.9317950458092976
Repeatition 1 Epoch 96 / 500 
 - time: 1.419360876083374 - sq_loss: 3.4658140066312626e-05 - tot_loss: 0.16778548177001085 - acc: 0.9427366702937976 - val_acc: 0.9321343739395996
Repeatition 1 Epoch 97 / 500 
 - time: 1.3590667247772217 - sq_loss: 3.3131414966192096e-05 - tot_loss: 0.1651759258936636 - acc: 0.9434167573449401 - val_acc: 0.9324737020699015
Repeatition 1 Epoch 98 / 500 
 - time: 1.285623550415039 - sq_loss: 3.169097180943936e-05 - tot_loss: 0.15994372833495163 - acc: 0.9443688792165397 - val_acc: 0.9351883271123176
Repeatition 1 Epoch 99 / 500 
 - time: 1.2732329368591309 - sq_loss: 3.0318775316118263e-05 - tot_loss: 0.1

Repeatition 1 Epoch 140 / 500 
 - time: 1.2429444789886475 - sq_loss: 9.229899660567753e-06 - tot_loss: 0.09315242474394836 - acc: 0.970076169749728 - val_acc: 0.9494401085850017
Repeatition 1 Epoch 141 / 500 
 - time: 1.2490947246551514 - sq_loss: 9.089393643080257e-06 - tot_loss: 0.08966992223707848 - acc: 0.970348204570185 - val_acc: 0.9491007804546997
Repeatition 1 Epoch 142 / 500 
 - time: 1.2300620079040527 - sq_loss: 8.954821169027127e-06 - tot_loss: 0.09170651703792032 - acc: 0.970620239390642 - val_acc: 0.9491007804546997
Repeatition 1 Epoch 143 / 500 
 - time: 1.2226660251617432 - sq_loss: 8.821052688290365e-06 - tot_loss: 0.09033332423734919 - acc: 0.970620239390642 - val_acc: 0.9491007804546997
Repeatition 1 Epoch 144 / 500 
 - time: 1.2660174369812012 - sq_loss: 8.701922524778638e-06 - tot_loss: 0.09034833141701881 - acc: 0.9707562568008705 - val_acc: 0.9487614523243977
Repeatition 1 Epoch 145 / 500 
 - time: 1.2599577903747559 - sq_loss: 8.586089279560838e-06 - tot_loss: 

Repeatition 1 Epoch 186 / 500 
 - time: 1.2212586402893066 - sq_loss: 6.145111001387704e-06 - tot_loss: 0.0710675093153732 - acc: 0.9794613710554951 - val_acc: 0.9552086868001357
Repeatition 1 Epoch 187 / 500 
 - time: 1.2243781089782715 - sq_loss: 6.114502411946887e-06 - tot_loss: 0.07082230359879205 - acc: 0.9794613710554951 - val_acc: 0.9552086868001357
Repeatition 1 Epoch 188 / 500 
 - time: 1.2293376922607422 - sq_loss: 6.08220079811872e-06 - tot_loss: 0.07198410282628842 - acc: 0.9797334058759521 - val_acc: 0.9555480149304377
Repeatition 1 Epoch 189 / 500 
 - time: 1.2258822917938232 - sq_loss: 6.051996933820192e-06 - tot_loss: 0.07053473959171086 - acc: 0.9797334058759521 - val_acc: 0.9555480149304377
Repeatition 1 Epoch 190 / 500 
 - time: 1.2253904342651367 - sq_loss: 6.022582510922803e-06 - tot_loss: 0.06933709599539029 - acc: 0.9798694232861807 - val_acc: 0.9555480149304377
Repeatition 1 Epoch 191 / 500 
 - time: 1.2233898639678955 - sq_loss: 5.9932799558737315e-06 - tot_los

...

Repeatition 1 Epoch 278 / 500 
 - time: 1.2235870361328125 - sq_loss: 4.5000292629993055e-06 - tot_loss: 0.05915650541076012 - acc: 0.985582154515778 - val_acc: 0.9592806243637597
Repeatition 1 Epoch 279 / 500 
 - time: 1.2339684963226318 - sq_loss: 4.488995273277396e-06 - tot_loss: 0.05538795451801448 - acc: 0.9857181719260065 - val_acc: 0.9592806243637597
Repeatition 1 Epoch 280 / 500 
 - time: 1.2212536334991455 - sq_loss: 4.4797502596338745e-06 - tot_loss: 0.055870027855249305 - acc: 0.9854461371055495 - val_acc: 0.9596199524940617
Repeatition 1 Epoch 281 / 500 
 - time: 1.2514348030090332 - sq_loss: 4.470084149943432e-06 - tot_loss: 0.05415501215862939 - acc: 0.985582154515778 - val_acc: 0.9596199524940617
Repeatition 1 Epoch 282 / 500 
 - time: 1.241487741470337 - sq_loss: 4.459436695469776e-06 - tot_loss: 0.05547401949359276 - acc: 0.985854189336235 - val_acc: 0.9592806243637597
Repeatition 1 Epoch 283 / 500 
 - time: 1.2227563858032227 - sq_loss: 4.448379968380323e-06 - tot_los

Repeatition 1 Epoch 324 / 500 
 - time: 1.2334909439086914 - sq_loss: 4.0438276300847065e-06 - tot_loss: 0.05483849529569618 - acc: 0.9874863982589771 - val_acc: 0.9613165931455717
Repeatition 1 Epoch 325 / 500 
 - time: 1.2353639602661133 - sq_loss: 4.034876837977208e-06 - tot_loss: 0.05388809941672967 - acc: 0.9874863982589771 - val_acc: 0.9609772650152698
Repeatition 1 Epoch 326 / 500 
 - time: 1.236842155456543 - sq_loss: 4.0267327676701825e-06 - tot_loss: 0.05315090214898177 - acc: 0.9874863982589771 - val_acc: 0.9606379368849678
Repeatition 1 Epoch 327 / 500 
 - time: 1.231215476989746 - sq_loss: 4.017878382001072e-06 - tot_loss: 0.05106918089120782 - acc: 0.9874863982589771 - val_acc: 0.9609772650152698
Repeatition 1 Epoch 328 / 500 
 - time: 1.2235352993011475 - sq_loss: 4.008482846984407e-06 - tot_loss: 0.05482889479731945 - acc: 0.9876224156692056 - val_acc: 0.9609772650152698
Repeatition 1 Epoch 329 / 500 
 - time: 1.2208607196807861 - sq_loss: 3.9992842175706755e-06 - tot_l

Repeatition 1 Epoch 370 / 500 
 - time: 1.2460644245147705 - sq_loss: 3.680769395941752e-06 - tot_loss: 0.05494824272952137 - acc: 0.9880304678998912 - val_acc: 0.9606379368849678
Repeatition 1 Epoch 371 / 500 
 - time: 1.2371625900268555 - sq_loss: 3.6733606521011097e-06 - tot_loss: 0.05361683711200094 - acc: 0.9880304678998912 - val_acc: 0.9606379368849678
Repeatition 1 Epoch 372 / 500 
 - time: 1.219374418258667 - sq_loss: 3.665293434096384e-06 - tot_loss: 0.057042502710118015 - acc: 0.9883025027203483 - val_acc: 0.9602986087546658
Repeatition 1 Epoch 373 / 500 
 - time: 1.244290828704834 - sq_loss: 3.6568503674061503e-06 - tot_loss: 0.05124627500945422 - acc: 0.9883025027203483 - val_acc: 0.9602986087546658
Repeatition 1 Epoch 374 / 500 
 - time: 1.223787784576416 - sq_loss: 3.65064556717698e-06 - tot_loss: 0.05372543558866738 - acc: 0.9881664853101197 - val_acc: 0.9606379368849678
Repeatition 1 Epoch 375 / 500 
 - time: 1.2199530601501465 - sq_loss: 3.645029892140883e-06 - tot_los

Repeatition 1 Epoch 416 / 500 
 - time: 1.2603552341461182 - sq_loss: 3.3926446576515445e-06 - tot_loss: 0.05252368245577266 - acc: 0.9891186071817193 - val_acc: 0.9619952494061758
Repeatition 1 Epoch 417 / 500 
 - time: 1.2683119773864746 - sq_loss: 3.3867872843984514e-06 - tot_loss: 0.04988822496542955 - acc: 0.9891186071817193 - val_acc: 0.9616559212758737
Repeatition 1 Epoch 418 / 500 
 - time: 1.2292306423187256 - sq_loss: 3.3807493764470564e-06 - tot_loss: 0.053747309910685104 - acc: 0.9893906420021763 - val_acc: 0.9619952494061758
Repeatition 1 Epoch 419 / 500 
 - time: 1.2561426162719727 - sq_loss: 3.373228764758096e-06 - tot_loss: 0.05409394979574422 - acc: 0.9893906420021763 - val_acc: 0.9616559212758737
Repeatition 1 Epoch 420 / 500 
 - time: 1.2220633029937744 - sq_loss: 3.3662147416180233e-06 - tot_loss: 0.04932406130324818 - acc: 0.9895266594124048 - val_acc: 0.9619952494061758
Repeatition 1 Epoch 421 / 500 
 - time: 1.2195301055908203 - sq_loss: 3.361734115969739e-06 - t

Repeatition 1 Epoch 462 / 500 
 - time: 1.245422601699829 - sq_loss: 3.1541821954306215e-06 - tot_loss: 0.05024249575145667 - acc: 0.9893906420021763 - val_acc: 0.9630132337970818
Repeatition 1 Epoch 463 / 500 
 - time: 1.285954236984253 - sq_loss: 3.1484269129578024e-06 - tot_loss: 0.048905031404544275 - acc: 0.9895266594124048 - val_acc: 0.9636918900576857
Repeatition 1 Epoch 464 / 500 
 - time: 1.2513461112976074 - sq_loss: 3.1410902465722756e-06 - tot_loss: 0.04884333635335736 - acc: 0.9895266594124048 - val_acc: 0.9636918900576857
Repeatition 1 Epoch 465 / 500 
 - time: 1.2341954708099365 - sq_loss: 3.137212161163916e-06 - tot_loss: 0.05078060117785732 - acc: 0.9895266594124048 - val_acc: 0.9636918900576857
Repeatition 1 Epoch 466 / 500 
 - time: 1.2686574459075928 - sq_loss: 3.1320378184318542e-06 - tot_loss: 0.051153717368921825 - acc: 0.9895266594124048 - val_acc: 0.9633525619273838
Repeatition 1 Epoch 467 / 500 
 - time: 1.230724811553955 - sq_loss: 3.126103365502786e-06 - tot